In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import random

from tqdm.notebook import tqdm

import plotly.express as px
import plotly.graph_objects as go

import sys; sys.path.append('../src/')
import one_dim_search.dichotomy as dichot
import one_dim_search.fib as fib
import one_dim_search.linear as lin
import one_dim_search.golden as gold
from descent.grad import gradient_descent_iter, get_constant_step_chooser

In [ ]:
def generate_func(*, cond: float, n: float):
    min_v = 1
    max_v = cond
    assert max_v / min_v == cond
    diag = [min_v, max_v]
    while len(diag) < n:
        diag.append(random.uniform(min_v, max_v))
    
#     m = np.diag(diag)
    b = np.random.randn(n)

    def f(arg):
        return np.sum(diag * arg ** 2 - b.dot(arg))

    def f_grad(arg):
        return diag * arg * 2 - b
    
    return f, f_grad, diag

In [ ]:
def analyze(step_chooser, eps=1e-3):
    data = []
    for cond in tqdm(np.linspace(1, 1000, 20)):
        for n in [5, 10, 20, 50, 100]:
            f, f_grad, _ = generate_func(cond=cond, n=n)
            it = gradient_descent_iter(
                f=f, f_grad=f_grad, eps=eps,
                start=np.random.randn(n),
                step_chooser=step_chooser,
                _verbose=1000
            )
            data.append({
                'cond': cond,
                'n': n,
                'cnt': sum(1 for _ in it)
            })
    fig = px.line(data, x='cond', y='cnt', color='n')
    fig.show()

In [ ]:
analyze(get_constant_step_chooser(1e-4), eps=0.01)

In [ ]:
def generic_step_chooser(one_dim_search):
    def step_chooser(f, x_k, cur_grad):
        phi = lambda h: f(x_k - h * cur_grad)
        l, r = lin.search(0, delta=1e-4, f=phi, eps=1e-5, multiplier=2)
        l, r = one_dim_search(l, r, f=phi, eps=1e-5)
        return (l + r) / 2
    return step_chooser

analyze(generic_step_chooser(gold.search))

In [ ]:
def create_matrix(condition_number, n):
    r = math.sqrt(condition_number)
    A = np.random.randn(n, n)
    u, s, v = np.linalg.svd(A)
    h, l = np.max(s), np.min(s)  # highest and lowest eigenvalues (h / l = current cond number)

    # linear stretch: f(x) = a * x + b, f(h) = h, f(l) = h/r, cond number = h / (h/r) = r
    def f(x):
        return h * (1 - ((r - 1) / r) / (h - l) * (h - x))

    new_s = f(s)
    new_A = (u * new_s) @ v.T  # make inverse transformation (here cond number is sqrt(k))
    new_A = new_A @ new_A.T  # make matrix symmetric and positive semi-definite (cond number is just k)
    assert np.isclose(np.linalg.cond(new_A), condition_number)
    return new_A


def number_of_iters(cond, n_vars, step_chooser, n_checks=100):
    avg_iters = 0
    for _ in range(n_checks):
        A = create_matrix(cond, n_vars)
        b = np.random.randn(len(A))
        init_x = np.random.randn(len(A))
        f = lambda x: x.dot(A).dot(x) - b.dot(x)
        f_grad = lambda x: (A + A.T).dot(x) - b

        # print(f(np.linalg.inv(A+A.T).dot(b))) -- optimal value

        trace = gradient_descent(f, f_grad, init_x, step_chooser, 'value')

        avg_iters += len(trace)
    return avg_iters / n_checks

In [ ]:
_, _, m = generate_func(cond=3, n=4)
m = np.diag(m)
print(m)
np.linalg.cond(m)

In [ ]:
m = create_matrix(3, 4)
print(m)
np.linalg.cond(m)

In [ ]:
def gen2(cond, n):
    A = create_matrix(cond, n)
    b = np.random.randn(n)
    def f(arg):
        return arg.dot(A).dot(x) - b.dot(x)
    return f

    def f_grad(arg):
        return diag * arg * 2 - b
    return f

    return f, f_grad, m

In [ ]:
def analyze2(step_chooser, eps=1e-3):
    data = []
    for cond in tqdm(np.linspace(1, 1000, 20)):
        for n in [5, 10, 20, 50, 100]:
            f, f_grad, _ = gen2(cond=cond, n=n)
            it = gradient_descent_iter(
                f=f, f_grad=f_grad, eps=eps,
                start=np.random.randn(n),
                step_chooser=step_chooser,
                _verbose=1000
            )
            data.append({
                'cond': cond,
                'n': n,
                'cnt': sum(1 for _ in it)
            })
    fig = px.line(data, x='cond', y='cnt', color='n')
    fig.show()

In [ ]:
def generic_step_chooser(one_dim_search):
    def step_chooser(f, x_k, cur_grad):
        phi = lambda h: f(x_k - h * cur_grad)
        l, r = lin.search(0, delta=1e-4, f=phi, eps=1e-5, multiplier=2)
        l, r = one_dim_search(l, r, f=phi, eps=1e-5)
        return (l + r) / 2
    return step_chooser

analyze(generic_step_chooser(gold.search))